<a href="https://colab.research.google.com/github/j100cky/AD_Sample_HDL_Size_Distribution_Analysis/blob/main/Assignment_Self_learning_tutorial_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting smoking time and life expectancy using the MIMIC-III clinical data and neural network

## Data acquisition and preprocessing

In [3]:
# RUN if files are located on your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

In [6]:
PATH_NOTEEVENTS = "/content/drive/MyDrive/MyUTAustin/2025_Summer_AIHealthcare/NOTEEVENTS.csv.gz"

df_noteevents = pd.read_csv(PATH_NOTEEVENTS, compression="gzip")
df_noteevents.info()

/tmp/ipython-input-6-3996132963.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_noteevents = pd.read_csv(PATH_NOTEEVENTS, compression="gzip")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [7]:
PATH_PATIENTS = "/content/drive/MyDrive/MyUTAustin/2025_Summer_AIHealthcare/PATIENTS.csv.gz"

df_patients = pd.read_csv(PATH_PATIENTS, compression="gzip")
df_patients.info()
df_patients.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46520 entries, 0 to 46519
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       46520 non-null  int64 
 1   SUBJECT_ID   46520 non-null  int64 
 2   GENDER       46520 non-null  object
 3   DOB          46520 non-null  object
 4   DOD          15759 non-null  object
 5   DOD_HOSP     9974 non-null   object
 6   DOD_SSN      13378 non-null  object
 7   EXPIRE_FLAG  46520 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [8]:
# Calculate age of patients
from datetime import datetime
df_patients_dead = df_patients
# Calculate age function
def calculate_age(dob, dod):
    dob_date = datetime.strptime(dob, "%Y-%m-%d %H:%M:%S")
    dod_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
    age = (dod_date - dob_date).days // 365
    return age

# Clean data of patients who are alive.
df_patients_dead = df_patients_dead.dropna(subset=['DOB', 'DOD'])

# Apply calculate_age function to create "age" column
df_patients_dead["age"] = df_patients_dead.apply(lambda row: calculate_age(row["DOB"], row["DOD"]), axis=1)

# Filter out rows with age greater than or equal to 120 or less than 10
df_patients_dead = df_patients_dead[(df_patients_dead["age"] < 120) | (df_patients_dead["age"] > 10)]

# Display the updated DataFrame
print(df_patients_dead.head())


    ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
1      235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
7      241         257      F  2031-04-03 00:00:00  2121-07-08 00:00:00   
10     244         261      M  2025-08-04 00:00:00  2102-06-29 00:00:00   
12     246         263      M  2104-06-18 00:00:00  2168-06-13 00:00:00   
17     251         268      F  2132-02-21 00:00:00  2198-02-18 00:00:00   

               DOD_HOSP              DOD_SSN  EXPIRE_FLAG  age  
1   2188-11-22 00:00:00                  NaN            1   23  
7   2121-07-08 00:00:00  2121-07-08 00:00:00            1   90  
10  2102-06-29 00:00:00  2102-06-29 00:00:00            1   76  
12  2168-06-13 00:00:00                  NaN            1   64  
17  2198-02-18 00:00:00                  NaN            1   66  


/tmp/ipython-input-8-4290838908.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_patients_dead["age"] = df_patients_dead.apply(lambda row: calculate_age(row["DOB"], row["DOD"]), axis=1)


Find patient noteevents with smoking noted.

In [9]:
# filter patients in the noteevents table that were dead.
df_noteevents_filtered = df_noteevents[df_noteevents['SUBJECT_ID'].isin(df_patients_dead['SUBJECT_ID'])]
df_noteevents_filtered.info()

del df_noteevents

<class 'pandas.core.frame.DataFrame'>
Index: 878333 entries, 1 to 2083091
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ROW_ID       878333 non-null  int64  
 1   SUBJECT_ID   878333 non-null  int64  
 2   HADM_ID      772520 non-null  float64
 3   CHARTDATE    878333 non-null  object 
 4   CHARTTIME    730302 non-null  object 
 5   STORETIME    485473 non-null  object 
 6   CATEGORY     878333 non-null  object 
 7   DESCRIPTION  878333 non-null  object 
 8   CGID         485473 non-null  float64
 9   ISERROR      448 non-null     float64
 10  TEXT         878333 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 80.4+ MB


Combine noteevents and patient tables by SUBJECT_ID

In [10]:
# Merge NOTEEVENTS and PATIENTS data on SUBJECT_ID
merged_data = (
    pd.merge(df_patients_dead, df_noteevents_filtered, on="SUBJECT_ID")
    .loc[:, ['SUBJECT_ID', 'age', 'TEXT']]  # Replace with your actual desired columns
)
merged_data.info()
merged_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878333 entries, 0 to 878332
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   SUBJECT_ID  878333 non-null  int64 
 1   age         878333 non-null  int64 
 2   TEXT        878333 non-null  object
dtypes: int64(2), object(1)
memory usage: 20.1+ MB


,SUBJECT_ID,age,TEXT
0,250,23,Admission Date: [**2188-11-12**] Discha...
1,250,23,PATIENT/TEST INFORMATION:\nIndication: Left ve...
2,250,23,Sinus tachycardia. Ventricular premature beats...


Select notes with the keyword smoking.

In [ ]:
# install packages for using BERT
!pip install transformers
!pip install torch

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
import torch

# Load the Clinical-BERT model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# Create a pipeline for NER Named Entity Recognition
ner = pipeline("ner", model=model, tokenizer=tokenizer)

In [17]:
# for each event note, check if it contains smoking, and if yes, find the years around the named entity.

import pandas as pd

# Sample data
data = {
    'EVENTNOTES': [
        "Patient has a history of smoking for 10 years.",
        "No history of smoking reported.",
        "The patient was advised to quit smoking after smoking for 5 years.",
        "Has been a smoker for over 20 years.",
        "Patient smokes intermittently, duration not specified."
    ]
}
df_events = pd.DataFrame(data)

# define keywords
smokeKeywords = ["smoking", "smoked", "smoker", "smokes", "smoke"]

# Function to extract smoking duration using simple string indexing
def extract_smoking_duration(text):
    #print(text)
    words = text.split()
    #print(words)
    for i in range(len(words)):
        #print(f'{i}: {words[i].lower()}')
        #print(f'word{i} is digit: {words[i].isdigit():}')
        #print(f'word{i}+1 is digit: {words[i+1].isdigit():}')
        #print(f'word{i}+2 is digit: {words[i+2].isdigit():}')
        if words[i].lower() in smokeKeywords:
          print(f'found {words[i]}')
          checkRange = 3
          checkRanges = pd.Series(range(-checkRange, checkRange + 1))
          for j in checkRanges:
            if i + j < 0 or i + j >= len(words):
              continue
            checkWord = words[i+j]
            #print(f'checkWord: {checkWord}')
            if checkWord.isdigit():
              print(f'found digit: {checkWord}')
              return float(checkWord)
            else:
              next
    return None  # No valid duration found

# Apply the function to the EVENTNOTES
df_events['smoking_duration'] = df_events['EVENTNOTES'].apply(extract_smoking_duration)

# View results
print(df_events[['EVENTNOTES', 'smoking_duration']])

found smoking
found digit: 10
found smoking
found smoking
found smoking
found digit: 5
found smoker
found digit: 20
found smokes
                                          EVENTNOTES  smoking_duration
0     Patient has a history of smoking for 10 years.              10.0
1                    No history of smoking reported.               NaN
2  The patient was advised to quit smoking after ...               5.0
3               Has been a smoker for over 20 years.              20.0
4  Patient smokes intermittently, duration not sp...               NaN


In [15]:
from tqdm import tqdm

In [16]:
# Enable tqdm for pandas
tqdm.pandas()

# Apply the function to the TEXT column
merged_data['smoking_duration(years)'] = merged_data['TEXT'].progress_apply(extract_smoking_duration)

# Display the updated DataFrame
#print(merged_data[['SUBJECT_ID', 'age', 'TEXT', 'smoking_duration']])
print(merged_data[['SUBJECT_ID', 'age', 'smoking_duration(years)']])

  0%|          | 1538/878333 [00:00<01:47, 8141.71it/s]

found smoking
found Smokes
found smoke
found Smokes
found smoke
found Smoker
found smoker
found smoker
found smoker
found smoker
found smoke
found smoking
found digit: 15
found smoke
found smoking
found digit: 20


  0%|          | 3910/878333 [00:00<01:23, 10499.16it/s]

found smoking
found smoking
found smoking
found smoker
found digit: 30
found smoking
found SMOKING
found digit: 5
found smoking
found digit: 30
found smoking
found digit: 50
found smoking
found smoking
found digit: 70
found smoking
found smoked
found smoking
found smoking
found digit: 30
found smoking
found smoking
found digit: 50
found SMOKES
found digit: 1


  1%|          | 6234/878333 [00:00<01:27, 9912.74it/s]

...

found smoking
found smoker
found smoker
found smoker
found Smoked
found smoking
found digit: 35
found smoker
found smoking
found smoking
found digit: 1
found Smoker
found smoker
found digit: 56
found smoking
found digit: 1
found smoking
found digit: 1
found Smoker
found smoking
found digit: 1
found smoking
found digit: 1
found smoking
found digit: 1
found smoking
found digit: 1
found smoking
found digit: 1
found smoking
found smoking
found digit: 1
found smoking
found digit: 1
found smoker
found Smoking
found smoking
found Smoking
found Smoking
found Smoking
found Smoking
found smoke
found Smoking
found smoke
found smoker
found smoking
found smoking
found Smoking
found Smoking
found smoking
found smoking
found Smoker
found Smoker
found smoking
found smoking
found smoking
found smoking
found digit: 40
found smoking
found smoking
found digit: 32
found smoking
found digit: 32
found smoking
found digit: 32
found smoking
found smoking
found smoking
found Smoked
found digit: 30
found smoker


100%|█████████▉| 876437/878333 [01:46<00:00, 12924.92it/s]

found smoker
found smoker
found smoker
found smoker
found smoking
found smokes
found smoking
found SMOKE
found smoke
found smoke
found smoke
found smoking
found smoking
found smokes
found smoking
found smoking
found smoke
found smoke
found smoke
found smoking
found digit: 5
found smoker
found smoking
found digit: 5
found smoker
found smoking
found smoker
found digit: 20
found smoker
found digit: 20
found smoker
found digit: 20
found smoker
found digit: 20
found smoker
found digit: 20
found smoker
found digit: 20
found Smoker
found smoker
found smoker
found smoking
found digit: 7
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found Smoker
found smoke
found smoker
found smoke
found digit: 1
found smoker
found smoke
found digit: 1
found smoker
found smoke
found digit: 1
found smoker
found smoke
found digit: 1
found Smoking
found smoke
found smoking
found smoking
found smoking
found digit: 2
fou

100%|██████████| 878333/878333 [01:46<00:00, 8244.46it/s] 

found smoking
found Smoked
found smoked
found digit: 35
found smoked
found smoker
found digit: 40
found smoker
found digit: 40
found smoked
found smoking
found smoking
found smoking
found smoking
found smoking
found smoked
found digit: 1
        SUBJECT_ID  age  smoking_duration(years)
0              250   23                      NaN
1              250   23                      NaN
2              250   23                      NaN
3              250   23                      NaN
4              250   23                      NaN
...            ...  ...                      ...
878328       44123   85                      NaN
878329       44123   85                      NaN
878330       44123   85                      NaN
878331       44123   85                      NaN
878332       44123   85                      NaN

[878333 rows x 3 columns]


In [19]:
# filter the merged_data to include only those with a smoking duration detected
merged_data_filtered = merged_data.dropna(subset=['smoking_duration(years)'])
merged_data_filtered.head()

,SUBJECT_ID,age,TEXT,smoking_duration(years)
876,704,84,Admission Date: [**2176-3-24**] Dischar...,15.0
1815,717,80,Admission Date: [**2116-2-4**] D...,20.0
2034,729,63,nursing admission note 2300-0700\n\nthis is a ...,30.0
2228,739,60,CTIC/SICU NURSING ADMISSION NOTE (MEDICAL BOAR...,5.0
2571,747,84,Admission Date: [**2147-3-7**] D...,30.0


In [20]:
# save the merged data for ease of use later
merged_data_filtered.to_csv('/content/drive/MyDrive/MyUTAustin/2025_Summer_AIHealthcare/deadPatientsWithSmokingDurationDefined.csv', index=False)

## Train a neural network model with data

In [21]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [22]:
# Read data from drive
data = pd.read_csv('/content/drive/MyDrive/MyUTAustin/2025_Summer_AIHealthcare/deadPatientsWithSmokingDurationDefined.csv')

In [58]:
# remove data with age values > 120
data = data[data['age'] < 120]

In [59]:
# Prepare features and target
X = data['smoking_duration(years)'].values.astype(np.float32).reshape(-1, 1)  # Features
y = data['age'].values.astype(np.float32).reshape(-1, 1)  # Target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Instantiate the model
model = SimpleNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Calculate loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if (epoch+1) % 10 == 0:  # Print every 10 epochs
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Show predictions compared to labeled values
print("Predictions:", test_outputs.numpy().flatten()[0:9])
print("Labeled Values:", y_test.flatten()[0:9])

Epoch [10/1000], Loss: 4580.9194
Epoch [20/1000], Loss: 3686.1360
Epoch [30/1000], Loss: 2720.6123
Epoch [40/1000], Loss: 2572.7454
Epoch [50/1000], Loss: 2495.4656
Epoch [60/1000], Loss: 2464.9856
Epoch [70/1000], Loss: 2427.4448
Epoch [80/1000], Loss: 2384.3669
Epoch [90/1000], Loss: 2326.5503
Epoch [100/1000], Loss: 2230.9258
Epoch [110/1000], Loss: 2099.5925
Epoch [120/1000], Loss: 1917.1031
Epoch [130/1000], Loss: 1667.7765
Epoch [140/1000], Loss: 1333.8364
Epoch [150/1000], Loss: 922.9106
Epoch [160/1000], Loss: 528.2283
Epoch [170/1000], Loss: 255.6990
Epoch [180/1000], Loss: 162.2200
Epoch [190/1000], Loss: 164.9276
Epoch [200/1000], Loss: 161.3645
Epoch [210/1000], Loss: 155.6675
Epoch [220/1000], Loss: 154.3314
Epoch [230/1000], Loss: 153.7540
Epoch [240/1000], Loss: 153.0344
Epoch [250/1000], Loss: 152.5505
Epoch [260/1000], Loss: 152.1708
Epoch [270/1000], Loss: 151.8924
Epoch [280/1000], Loss: 152.2487
Epoch [290/1000], Loss: 151.6768
Epoch [300/1000], Loss: 151.3999
Epoch

In [60]:
print("Predictions:", test_outputs.numpy().flatten()[0:9])
print("Labeled Values:", y_test.flatten()[0:9])

Predictions: [70.30087  63.054028 63.054028 71.10478  68.69309  71.90869  73.51647
 72.1555   68.69309 ]
Labeled Values: [62. 47. 71. 78. 67. 70. 53. 54. 80.]


In [61]:
# test the model with imaginary smoking duration
print(f'Expected age for someone who smoked for 10 years: {model(torch.tensor([[10.0]]))}')
print(f'Expected age for someone who smoked for 20 years: {model(torch.tensor([[20.0]]))}')
print(f'Expected age for someone who smoked for 5 years: {model(torch.tensor([[5.0]]))}')
print(f'Expected age for someone who smoked for 1 years: {model(torch.tensor([[1.0]]))}')
print(f'Expected age for someone who smoked for 60 years: {model(torch.tensor([[60.0]]))}')


Expected age for someone who smoked for 10 years: tensor([[68.6931]], grad_fn=<AddmmBackward0>)
Expected age for someone who smoked for 20 years: tensor([[70.3009]], grad_fn=<AddmmBackward0>)
Expected age for someone who smoked for 5 years: tensor([[67.4602]], grad_fn=<AddmmBackward0>)
Expected age for someone who smoked for 1 years: tensor([[63.0540]], grad_fn=<AddmmBackward0>)
Expected age for someone who smoked for 60 years: tensor([[72.6361]], grad_fn=<AddmmBackward0>)
